# MVSep-MDX23 Colab Fork v2.4
Адаптация алгоритма MVSep-MDX23 для Google Colab, с различными надстройками:

https://colab.research.google.com/github/alphatoasterous/MVSEP-MDX23-Colab_v2/blob/v2.4/MVSep-MDX23-Colab.ipynb

<details>
    <summary>Список изменений:</summary>
<font size=2>

**v2.4**
* BS-Roformer models from viperx added
* MDX-InstHQ4 model added as optional
* Flac output
* Control input volume gain
* Filter vocals below 50Hz option
* Better chunking algo (no clicks)
* Some code cleaning

</font>
<br>

<br>
<font size=2>
<br>

[**v2.3**](https://github.com/alphatoasterous/MVSEP-MDX23-Colab_v2/tree/v2.3)
* HQ3-Instr model replaced by VitLarge23 (thanks to MVSep)
* Improved MDXv2 processing (thanks to Anjok)
* Improved BigShifts algo (v2)
* BigShifts processing added to MDXv3 & VitLarge
* Faster folder batch processing

[**v2.2.2**](https://github.com/alphatoasterous/MVSEP-MDX23-Colab_v2/tree/v2.2)
* Improved MDXv3 chunking code (thanks to HymnStudio)
* D1581 demo model replaced by new InstVocHQ MDXv3 model.
<br>

**v2.2.1**
* Added custom weights feature
* Fixed some bugs
* Fixed input: you can use a file or a folder as input now
<br>

**v2.2**
* Added MDXv3 compatibility
* Added MDXv3 demo model D1581 in vocals stem multiband ensemble.
* Added VOC-FT Fullband SRS instead of UVR-MDX-Instr-HQ3.
* Added 2stems feature : output only vocals/instrum (faster processing)
* Added 16bit output format option
* Added "BigShift trick" for MDX models
* Added separated overlap values for MDX, MDXv3 and Demucs
* Fixed volume compensation fine-tuning for MDX-VOC-FT
<br>

[**v2.1 (by deton24)**](https://github.com/deton24/MVSEP-MDX23-Colab_v2.1)
* Updated with MDX-VOC-FT instead of Kim Vocal 2
<br>

[**v2.0**](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/tree/2.0)
* Updated with new Kim Vocal 2 & UVR-MDX-Instr-HQ3 models
* Folder batch processing
* Fixed high frequency bleed in vocals
* Fixed volume compensation for MDX models
<br>
</font>
</details>
<br>

Credits:
* [ZFTurbo/MVSep](https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model)
* Модели: [Demucs](https://github.com/facebookresearch/demucs), [Anjok](https://github.com/Anjok07/ultimatevocalremovergui) & [Kimberley Jensen](https://github.com/KimberleyJensen)
* Адаптация и надстройки: [jarredou](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/)
* Перевод на русский и интерактивная загрузка: [alphatoaster](https://github.com/alphatoasterous/MVSEP-MDX23-Colab_v2/)
</font>

In [ ]:
#@markdown #Установка
#@markdown *Запустите эту ячейку для установки MVSep-MDX23. Согласитесь дать доступ к Google Drive в всплывающем окне!*<br>
#@markdown ⚠️ Установка делается каждую новую сессию(новую вкладку в браузере) Google Colab! ⚠️
print('Начинаем установку... Она может занять до 15 минут, из-за достаточно тормознутого Колаба. :(')
%cd /content
from google.colab import drive
drive.mount('/content/drive')
!git clone -b v2.4 https://github.com/alphatoasterous/MVSEP-MDX23-Colab_v2  &> /dev/null
%cd /content/MVSEP-MDX23-Colab_v2 
print('Устанавливаем зависимости...')
!pip install -r requirements.txt &> /dev/null
# onnxruntime-gpu nightly fix for cuda12.2
print('Устанавливаем фикс onnxruntime для CUDA 12.x...')
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/  &> /dev/null
print('Установка завершена!')

In [ ]:
#@markdown #Загрузка аудио
#@markdown *Запустите эту ячейку для загрузки аудиофайлов (WAV, FLAC, MP3 или zip-файлов, содержащих аудиофайлы) в этот Colab.*

from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

# Задать директорию вывода, создать если её нет
target_dir = '/content/audio_files/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    # Если файл - zip-архив
    if fn.endswith('.zip'):
        # Записать архив
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4])  # Remove the .zip extension from the folder name

        # Распаковать архив
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        # Удалить архив
        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Zip-архив "{name}" разархивирован. Файлы находятся в: {folder}'.format(name=fn, folder=unzip_dir))

        # Вывести кнопку ссылки для каждого файла
        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Разархивирован файл "{name}" с размером {length} байт'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='Файл сохранен: <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    # Если файл - не zip-архив
    # Сохранить файл в директорию
    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='Пользователь загрузил файл "{name}" с размером {length} байт'.format(name=fn, length=file_length)
    )
    display(file_label)

    # Проверить, если файл поддерживаемого формата: .wav, .mp3, .flac
    if not (fn.endswith('.wav') or fn.endswith('.mp3') or fn.endswith('.flac')):
        warning_text = widgets.HTML(
            value='<b style="color: red;">ВНИМАНИЕ:</b> Вы загружаете неподдерживаемый файл.'
        )
        display(warning_text)

    # Создать кликабельную ссылку на файл
    file_path_text = widgets.HTML(
        value='Файл сохранен: <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path  # Make a local copy of the file path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

### О параметрах:

Стандартная конфигурация оптимально работает на бесплатном T4 инстансе Google Colab.
Двигайте ползунки на свой страх и риск!


<font size=2>

* **BigShifts:** Качество/скорость лучше при значениях от 3 до 11, **НО** 11 не всегда дает наилучшие результаты. Работает примерно как seed: разные значения дают разные результаты.<br>
Чем больше значение, тем дольше обработка.
</font>

<font size=2>

* **Overlap InstVoc/VitLarge:** Нет преимущества в использовании высоких значений, если BigShifts на дефолтном значении. Если BigShifts=1 (обычная обработка), используйте более высокие значения, например 8 или даже 16.<br>
Чем больше значение, тем дольше обработка.<br>
 *То же самое относится и к overlap_VOCFT, но со значениями от 0 до 0.95.*
</font>

<font size=2>

* **Use VOCFT:** Определяет, использовать VOCFT или нет. <br>
Обработка дольше, качество *возможно* выше.<br>
</font>

<font size=2>

* **Weights:** Взвешивание моделей: насколько приоритетнее результат обработки данной модели для вырезаемого трека.
</font>


In [ ]:
#@markdown #Разделение трека
from pathlib import Path
import glob

%cd /content/MVSEP-MDX23-Colab_v2

#@markdown ---
#@markdown #### Конфигурация:
#@markdown Скопируйте путь, полученный в ячейке загрузки, если аудио было загружено через неё!
#@markdown Обычно все загруженные через загрузчик файлы хранятся в /content/audio_files
input = '/content/drive/MyDrive/input' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}

output_format = 'FLAC' #@param ["PCM_16", "FLOAT", "FLAC"]
Separation_mode = 'Vocals/Instrumental' #@param ["Vocals/Instrumental", "4-STEMS"]
input_gain = 0 #@param [0, -3, -6] {type:"raw"}
restore_gain_after_separation = False #@param {type:"boolean"}
filter_vocals_below_50hz = False #@param {type:"boolean"}
#@markdown ___
##@markdown

#@markdown  ### Конфигурация моделей:

#@markdown  *Bigshifts=1 отключает BigShifts*
BigShifts = 3 #@param {type:"slider", min:1, max:41, step:1}
#@markdown ---
BSRoformer_model = 'ep_317_1297' #@param ["ep_317_1297", "ep_368_1296"]
weight_BSRoformer = 10 #@param {type:"slider", min:0, max:10, step:1}
##@markdown ---
weight_InstVoc = 4 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
use_VitLarge = True #@param {type:"boolean"}
weight_VitLarge = 1 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
use_InstHQ4 = False #@param {type:"boolean"}
weight_InstHQ4 = 2 #@param {type:"slider", min:0, max:10, step:1}
overlap_InstHQ4 = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
use_VOCFT = False #@param {type:"boolean"}
weight_VOCFT = 2 #@param {type:"slider", min:0, max:10, step:1}
overlap_VOCFT = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
#@markdown  *Demucs используется только в режиме разделения на несколько сурсов*
overlap_demucs = 0.6 #@param {type:"slider", min:0, max:0.95, step:0.05}

use_InstVoc_ = '--use_InstVoc' #forced use
use_BSRoformer_ =  '--use_BSRoformer' #forced use
use_VOCFT_ = '--use_VOCFT' if use_VOCFT is True else ''
use_VitLarge_ = '--use_VitLarge' if use_VitLarge is True else ''
use_InstHQ4_ = '--use_InstHQ4' if use_InstHQ4 is True else ''
restore_gain = '--restore_gain' if restore_gain_after_separation is True else ''
vocals_only = '--vocals_only' if Separation_mode == 'Vocals/Instrumental' else ''
filter_vocals = '--filter_vocals' if filter_vocals_below_50hz is True else ''

if Path(input).is_file():
  file_path = input
  Path(output_folder).mkdir(parents=True, exist_ok=True)
  !python inference.py \
        --input_audio "{file_path}" \
        --large_gpu \
        --BSRoformer_model {BSRoformer_model} \
        --weight_BSRoformer {weight_BSRoformer} \
        --weight_InstVoc {weight_InstVoc} \
        --weight_InstHQ4 {weight_InstHQ4} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_VitLarge {weight_VitLarge} \
        --overlap_demucs {overlap_demucs} \
        --overlap_VOCFT {overlap_VOCFT} \
        --overlap_InstHQ4 {overlap_InstHQ4} \
        --output_format {output_format} \
        --BigShifts {BigShifts} \
        --output_folder "{output_folder}" \
        --input_gain {input_gain} \
        {filter_vocals} \
        {restore_gain} \
        {vocals_only} \
        {use_VitLarge_} \
        {use_VOCFT_} \
        {use_InstHQ4_} \
        {use_InstVoc_} \
        {use_BSRoformer_}


else:
    file_paths = sorted(glob.glob(input + "/*"))[:]
    input_audio_args = ' '.join([f'"{path}"' for path in file_paths])
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    !python inference.py \
        --input_audio {input_audio_args} \
        --large_gpu \
        --BSRoformer_model {BSRoformer_model} \
        --weight_BSRoformer {weight_BSRoformer} \
        --weight_InstVoc {weight_InstVoc} \
        --weight_InstHQ4 {weight_InstHQ4} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_VitLarge {weight_VitLarge} \
        --overlap_demucs {overlap_demucs} \
        --overlap_VOCFT {overlap_VOCFT} \
        --overlap_InstHQ4 {overlap_InstHQ4} \
        --output_format {output_format} \
        --BigShifts {BigShifts} \
        --output_folder "{output_folder}" \
        --input_gain {input_gain} \
        {filter_vocals} \
        {restore_gain} \
        {vocals_only} \
        {use_VitLarge_} \
        {use_VOCFT_} \
        {use_InstHQ4_} \
        {use_InstVoc_} \
        {use_BSRoformer_}